In [0]:
!mkdir dataset
from google.colab import files
src = list(files.upload().values())[0]
open('\dataset','wb').write(src)
!mv sample_mobile_data_1000.csv dataset/
!ls dataset

In [0]:
!curl ifconfig.co

34.76.163.135


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/My\ Drive/
!cp -r /content/gdrive/My\ Drive/BO-* /content/gdrive/My\ Drive/resume
!cd gdrive/My\ Drive/colab

 BO-DA-رزومه‌ها-E0		  nimac.gsite
 colab				  SahabX-Sokan
'Getting started.pdf'		 'اعضای دانشکده.xlsx'
'hadoop meeting feedback.gform'  'پروژه هوش سحاب.gdoc'
'Hadoop Meeting.gslides'	 'مشخصات سرور.gsheet'
 NimaBehrang.pdf


In [0]:
!unzip /content/gdrive/My\ Drive/colab/ml-100k.zip

In [0]:
!ls gdrive/My\ Drive/colab/ml-100k/

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [19]:
# !cat gdrive/My\ Drive/colab/ml-100k/u.user
import numpy as np
data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data", dtype=int)
# data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data",  dtype=[int,int,int,int])
train_data = []
test_data = []
users = np.unique(data[:,0])
for user in users:
  userrows = data[(np.where(data[:,0] == user)), :][0]
  np.random.shuffle(userrows)
  for item in userrows[:20]:
    train_data.append(item.tolist())
  for item in userrows[20:]:
    test_data.append(item.tolist())

print(len(train_data))
print(np.shape(train_data))


27051
(27051, 4)


In [0]:
!ls 

In [3]:
!pip install -q tensorflow-gpu==2.0.0a0
!pip install -q tensorboard
!pip install -q tensorboardcolab

     |████████████████████████████████| 332.1MB 63kB/s 
     |████████████████████████████████| 419kB 40.5MB/s 
     |████████████████████████████████| 3.0MB 52.7MB/s 


In [4]:
from tensorboardcolab import *
tbc=TensorBoardColab(port=6006, graph_path='/tmp/logs', startup_waiting_time=8)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://ee7543b3.ngrok.io


In [0]:
def castRate(inp):
  if inp == 1:
    return 0.1
  if inp == 2:
    return 0.3
  if inp == 3:
    return 0.5
  if inp == 4:
    return 0.7
  if inp == 5:
    return 1

In [0]:
from tensorflow.python import keras
import tensorflow as tf

In [0]:
keras.backend.clear_session()

In [0]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
multiply = keras.layers.Multiply()([embed_user, embed_item])
rate = keras.layers.add(tf.unstack(multiply, axis=2))
model = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(rate))
# model.summary()

In [58]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
dot = keras.layers.dot(inputs=[embed_user, embed_item],axes=2)
model_opt = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(dot))
model_opt.summary()
sgd = tf.keras.optimizers.SGD(lr=0.01, clipvalue=0.5)
adam = tf.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
nadam = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
square = tf.keras.losses.squared_hinge
ms = 'mean_squared_error'
model_opt.compile(optimizer=nadam,
            loss=ms,
            metrics=['accuracy','mae'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 64)        128000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 64)        128000      input_2[0][0]                    
______________________________________________________________________________________________

In [84]:
train = np.asarray(train_data)
test = np.asarray(test_data)
x_train = [train[:,0],train[:,1]]
y_train = [castRate(x) for x in train[:,2]]
x_test = [test[:,0],test[:,1]]
y_test = [castRate(x) for x in test[:,2]]
test_sorted = np.asarray(sorted(test, key=lambda x: x[2], reverse=True))
x_test_sorted = [test_sorted[:,0],test_sorted[:,1]]
y_test_sorted = [castRate(x) for x in test_sorted[:,2]]

logdir="/tmp/logs/2"
checkpoint_path = logdir + "/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_path)

# fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10,
fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30,
                callbacks = [cp_callback, tb_callback], batch_size=1000)

# ans = model_opt.evaluate(x_test, y_test)

Train on 27051 samples, validate on 72949 samples


W0714 19:10:58.163105 139765835683712 tf_logging.py:161] Model failed to serialize as JSON. Ignoring... ('Not JSON Serializable:', b'\n\nSigmoid_20\x12\x07Sigmoid\x1a\rdot_10/MatMul*\x07\n\x01T\x12\x020\x01')


Epoch 1/30
18000/27051 [==================>...........] - ETA: 0s - loss: 0.0658 - accuracy: 0.2321 - mae: 0.2082
Epoch 00001: saving model to /tmp/logs/2/cp.ckpt
27051/27051 [==============================] - 0s 14us/sample - loss: 0.0653 - accuracy: 0.2325 - mae: 0.2071 - val_loss: 0.0650 - val_accuracy: 0.2031 - val_mae: 0.2068
Epoch 2/30
22000/27051 [=======================>......] - ETA: 0s - loss: 0.0651 - accuracy: 0.2316 - mae: 0.2065
Epoch 00002: saving model to /tmp/logs/2/cp.ckpt
27051/27051 [==============================] - 0s 13us/sample - loss: 0.0653 - accuracy: 0.2325 - mae: 0.2069 - val_loss: 0.0650 - val_accuracy: 0.2031 - val_mae: 0.2069
Epoch 3/30
23000/27051 [========================>.....] - ETA: 0s - loss: 0.0654 - accuracy: 0.2333 - mae: 0.2071
Epoch 00003: saving model to /tmp/logs/2/cp.ckpt
27051/27051 [==============================] - 0s 13us/sample - loss: 0.0653 - accuracy: 0.2324 - mae: 0.2070 - val_loss: 0.0651 - val_accuracy: 0.2031 - val_mae: 0.2067
E

In [85]:
predict = model_opt.predict(x_test_sorted)
for i in range(10):
  print(predict[i], y_test[i])

[[0.6322765]] 0.3
[[0.634879]] 0.7
[[0.6356948]] 0.5
[[0.6369672]] 0.7
[[0.646109]] 0.1
[[0.62895405]] 0.5
[[0.6344369]] 0.7
[[0.6271061]] 0.5
[[0.6432345]] 1
[[0.6406243]] 1


In [90]:
def calcHitRatio(x_test, y_test):
  ans = []
  for indx, item in enumerate(predict):
    ans.append([indx, item])
  ans.sort(key=lambda x: x[1],reverse=True)
  tresh = 100
  count = 0
  for i in range(tresh):
      if ans[i][0] <= tresh:
        count = count + 1
  return count
print(calcHitRatio(x_test_sorted, y_test_sorted))


0
